# sklearn.compose

En Scikit-Learn, el **módulo sklearn.compose** proporciona herramientas que facilitan la construcción de flujos de trabajo más complejos al combinar diferentes transformaciones en un solo objeto. Es útil cuando necesitas aplicar múltiples transformaciones a diferentes columnas de un conjunto de datos o cuando deseas aplicar una secuencia de transformaciones de manera flexible y estructurada.
Incluye dos clases principales:
- ColumnTransformer
- TransformedTargetRegressor (menos utilizada)

## ColumnTransformer

La clase **ColumnTransformer** permite aplicar transformaciones diferentes a distintas columnas de un conjunto de datos. Es útil cuando en los datos aparecen diferentes tipos de variables (por ejemplo, numéricas y categóricas) y necesitas aplicar distintas transformaciones a cada tipo de columna. 

•	El *ColumnTransformer* toma una lista de transformaciones y aplica una transformación específica a las columnas que le indiques.

•	Se pueden aplicar diferentes preprocesamientos (normalización, codificación de variables categóricas, ...) a diferentes subconjuntos de las columnas de un DataFrame.

Supongamos que tienes un conjunto de datos con columnas numéricas y categóricas, y deseas aplicar diferentes transformaciones a cada tipo de columna

In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


df = pd.DataFrame({
    'edad': [25, 30, np.nan, 35, 40],
    'genero': ['M', 'F', 'M', 'F', 'M'],
    'ingresos': [50000, 60000, 70000, 80000, np.nan]
})

preprocessor = ColumnTransformer(
    transformers=[                  #formato: ('nombre',transformador,columnas)
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), ['edad', 'ingresos']),
        
        ('cat', OneHotEncoder(), ['genero'])
    ]
)

print(df)

df_transformed = preprocessor.fit_transform(df)

print(df_transformed)

num_cols = ['edad', 'ingresos']

# Nombres de las columnas categóricas después del OneHotEncoder
cat_cols = preprocessor.transformers_[1][1].get_feature_names_out(['genero'])

# Combinar los nombres de las columnas
columns = np.concatenate([num_cols, cat_cols])


df_transformed_final = pd.DataFrame(df_transformed, columns=columns)


print(df_transformed_final)


   edad genero  ingresos
0  25.0      M   50000.0
1  30.0      F   60000.0
2   NaN      M   70000.0
3  35.0      F   80000.0
4  40.0      M       NaN
[[-1.5 -1.5  0.   1. ]
 [-0.5 -0.5  1.   0. ]
 [ 0.   0.5  0.   1. ]
 [ 0.5  1.5  1.   0. ]
 [ 1.5  0.   0.   1. ]]
   edad  ingresos  genero_F  genero_M
0  -1.5      -1.5       0.0       1.0
1  -0.5      -0.5       1.0       0.0
2   0.0       0.5       0.0       1.0
3   0.5       1.5       1.0       0.0
4   1.5       0.0       0.0       1.0


Se aplica un SimpleImputer (para reemplazar valores faltantes) y normalización (con StandardScaler) a las columnas numéricas ('edad' y 'ingresos').

Se aplica codificación one-hot (OneHotEncoder) a la columna categórica ('genero').

- Pipeline:

Dentro del ColumnTransformer, se usa un Pipeline para encadenar múltiples pasos (primero imputación y luego normalización para las columnas numéricas).

Utilizamos fit_transform() porque el preprocesador primero tiene que aprender (fit) la información de las columnas del Dataframe, en este caso la media, la desviación para la normalización y aprender los valores de la columna categórica(genero) y después aplica (transform) esas transformaciones que acaba de aprender a los datos, es decir, pondrá los valores nulos con el valor de la media y luego las normaliza con la desviación estándar que aprendió. En la columna categórica convertirá las categorías a valores binarios. 

Por cierto, *'num'* y *'cat'* son solo etiquetas que pueden tener cualquier valor siempre que sea un string, no influyen en el código, solo se utilizan para hacer más claro para qué son las transformaciones

En cuanto a: *preprocessor.transformers_[1][1].get_feature_names_out(['genero'])* . Cuando defines un ColumnTransformer tiene el formato *('nombre',transformador,columnas)* con lo que ya que estoy accediendo a cat será el indíce [1] y el segundo [1] se refiere al transformador dentro de esa tupla que es el OneHotEncoder, así que al acceder a él puedo llamar a sus métodos, en este caso, get_feature_names_out que me devuelve los nombres de las nuevas características generadas